In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
import spacy

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import nltk
from nltk.corpus import stopwords
import re
import string

# gensim
from gensim import corpora, models, similarities, matutils

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from os import path
from PIL import Image
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from corextopic import corextopic as ct
from corextopic import vis_topic as vt

/Users/ginachung/anaconda3/lib/python3.6/site-packages/corextopic/vis_topic.py:9: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/ginachung/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/ginachung/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/ginachung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/ginachung/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/ginachung/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp

In [2]:
tng = pd.read_csv("/Users/ginachung/Desktop/TNG.csv")

In [3]:
tng.head()

,0,episode,productionnumber,setnames,characters,act,scenenumber,scenedetails,partnumber,type,who,text,speechdescription,Released,Episode,imdbRating,imdbID,Season
0,1,encounter at farpoint,NaN,NaN,NaN,TEASER,1,NaN,1,description,NaN,The U.S.S. Enterprise NCC 1701-D traveling at...,False,1987-09-26,1.0,6.9,tt0094030,1.0
1,2,encounter at farpoint,NaN,NaN,NaN,TEASER,1,NaN,2,speech,PICARD V.O.,"Captain's log, stardate 42353.7. Our destinat...",False,1987-09-26,1.0,6.9,tt0094030,1.0
2,3,encounter at farpoint,NaN,NaN,NaN,TEASER,2,NaN,3,description,NaN,on the gigantic new Enterprise NCC 1701-D.,False,1987-09-26,1.0,6.9,tt0094030,1.0
3,4,encounter at farpoint,NaN,NaN,NaN,TEASER,2,NaN,4,speech,PICARD V.O.,"My orders are to examine Farpoint, a starbase...",False,1987-09-26,1.0,6.9,tt0094030,1.0
4,5,encounter at farpoint,NaN,NaN,NaN,TEASER,3,NaN,5,description,NaN,"Huge, with a giant wall diagram showing the i...",False,1987-09-26,1.0,6.9,tt0094030,1.0


In [ ]:
tng.isnull().sum()

In [ ]:
tngspeech = tng[tng.type != 'description']

In [ ]:
tngspeech.head(20)

In [ ]:
tng.isnull().sum()

In [ ]:
tngspeech=tng.drop(columns=['0','productionnumber', 'setnames','characters','act','scenedetails','speechdescription','imdbID' ],inplace=True)

In [ ]:
tngspeech=tng

In [ ]:
tngspeech = tngspeech[tngspeech.type != 'description']

In [ ]:
tngspeech.isnull().sum()

In [ ]:
tngspeech.head(20)

In [ ]:
tngspeech['who']=tngspeech['who'].str.strip()

In [ ]:
tngspeech['who']=tngspeech['who'].str.replace('\'S COM VOICE','')

In [ ]:
tngspeech['who'].value_counts()

In [ ]:
tngspeech['post']=tngspeech['text'].str.replace('[{}]'.format(string.punctuation),'')

In [ ]:
tngspeech['post']=tngspeech['post'].str.replace('\d+'.format(string.punctuation),'')

In [ ]:
tngspeech['who']=tngspeech['who'].str.replace('PICARD (V.O.)','PICARD V.O.')
tngspeech['who']=tngspeech['who'].str.replace('PICARD V.O.','PICARD')

In [ ]:
tngspeech['post']=tngspeech['post'].str.lower()

In [ ]:
tngspeech['who']=tngspeech['who'].str.replace('PICARD (V.O.)',"PICARD" )

In [ ]:
tngspeech['who']=tngspeech['who'].str.strip()

In [ ]:
tngspeech

In [ ]:
tngspeech['who'].value_counts()

#TOPIC Modelling

In [ ]:
my_list =['understand','trying','learn','full','great','wrong' ,'knew','old','keep','new','isnt','part','remember','things','kind','wanted','someone','mean','possible','someone','found','leave','really','cannot','happened','last','feel','every','many','always','vessel','said','wont','ago','youll','doesnt','well','thing','get','another','three','know','im','dont','us','would','get','take','back','let','give','first','nine','nothing','would','energy','like', 'us','well','system',"worf",'able',"starfleet","sorry","eight","help","enterprise","perhaps","look","made","give","lets","let","better","new","youve","enough","left","try","id","sure","four","doctor","years","hundred","didnt","point","much","work",'hes',"good","might","please","human","ready","still","whats","man","weve","long","people","even","nothing","ever","thank","ship","maybe","find","come","got","anything","thought","would","believe","cant","little","another","never","theres","two","three","five",'think',"back","think","lieutenant",'right',"must","first",'need',"im","la","mister","go",'tell','say','something','take','could','theyre','yes','ive','ill','make','time','may','going','way','thats','dont','know','youre','get','see','want','thing','life']

In [ ]:
stop_english= stopwords.words('english')

In [ ]:
stop_english.extend(my_list)

In [ ]:
cv = CountVectorizer(stop_words=stop_english,max_df=0.02)

X=cv.fit_transform(tngspeech['post'])

In [ ]:
X.shape

In [ ]:
import pandas as pd

pd.DataFrame(X.toarray(), index=tngspeech['post'], columns=cv.get_feature_names()).head(10)

In [ ]:
nmf_model = NMF(10)
doc_topic = nmf_model.fit_transform(X)

In [ ]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5",'component_6','component_7','component_8','component_9','component_10'],
             columns = cv.get_feature_names())
topic_word

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
  #  nlist=[]
   # llist=[]
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
nmf_model.components_

In [ ]:
#5 TOPICS FOR THE WHOLE SERIES
display_topics(nmf_model, cv.get_feature_names(), 10)

In [ ]:
nmf_model

In [ ]:
H = pd.DataFrame(doc_topic.round(5),
tngspeech['post'],
             columns = ["component_1","component_2",'component_3','component_4','component_5','component_6','component_7','component_8','component_9','component_10' ])
H

In [ ]:
len(H)

In [ ]:
H['topic'] = H[["component_1","component_2",'component_3','component_4','component_5','component_6','component_7','component_8','component_9','component_10' ]].idxmax(axis=1)

In [ ]:
tngspeech['topic'] = list(H.topic)

In [ ]:
tngspeech.head()

In [ ]:
tngspeech.head()

In [ ]:
tngspeech['topic']=tngspeech['topic'].str.replace('component_10','sickbay')

In [ ]:
tngspeech['topic']=tngspeech['topic'].str.replace('component_1','logentry')

In [ ]:
tngspeech['topic']=tngspeech['topic'].str.replace('component_2','greeting')

In [ ]:
tngspeech['topic']=tngspeech['topic'].str.replace('component_3','bridge')

In [ ]:
tngspeech['topic']=tngspeech['topic'].str.replace('component_4','awayteam')

In [ ]:
tngspeech['topic']=tngspeech['topic'].str.replace('component_5','engineering')

In [ ]:
tngspeech['topic']=tngspeech['topic'].str.replace('component_6','computer')

In [ ]:
tngspeech['topic']=tngspeech['topic'].str.replace('component_7','transporterroom')

In [ ]:
tngspeech['topic']=tngspeech['topic'].str.replace('component_8','characters')

In [ ]:
tngspeech['topic']=tngspeech['topic'].str.replace('component_9','shipstatus')

In [ ]:
tngspeech.info()

In [ ]:
tngspeech['post']

In [ ]:
filter1=tngspeech.groupby(['Season','topic'])['who'].value_counts()

In [ ]:
filter1 = filter1.reset_index(name='count')

In [ ]:
filter1.sort_values(['Season','topic','count'],ascending=False).groupby(['Season','topic']).head(8)

In [ ]:
filter2=tngspeech.groupby(['Season'])['who'].value_counts()

In [ ]:
filter2 = filter2.reset_index(name='count')
filter2.sort_values(['Season','count'],ascending=False).groupby(['Season']).head(8)

In [ ]:
from IPython.display import display

def show_more(df, lines):
    foo = 1
    display(df)
    foo = 2   

In [ ]:
with pd.option_context('display.max_rows', 999):
    print (filter1)   

In [ ]:
mask=(tngspeech['who']=='PICARD')

In [ ]:
tngpicard = tngspeech[mask]

In [ ]:
#tngpicard
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
gather=tngspeech.groupby('Season')['topic'].value_counts(normalize=True)
gather.plot(kind='bar')
plt.show()

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
def display_topics1(model, feature_names, no_top_words, topic_names=None):
  #  nlist=[]
    llist=[]
    for ix,topic in enumerate(model.components_):
       # if not topic_names or not topic_names[ix]:
           #
            #print("\nTopic ")
        #else:
        #print("\nTopic: '",topic_names,"'")
        llist.append(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    return llist

In [ ]:
list1=display_topics1(nmf_model, cv.get_feature_names(), 10)

In [ ]:
# def display_topics1(model, feature_names, no_top_words, topic_names=None):
#   #  nlist=[]
#     llist=[]
#     for ix,topic in enumerate(model.components_):
#        # if not topic_names or not topic_names[ix]:
#            #
#             #print("\nTopic ")
#         #else:
#         #print("\nTopic: '",topic_names,"'")
#         print(" ".join([feature_names[i]
#                         for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
display_topics1(nmf_model, cv.get_feature_names(), 10)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS 

text = 'sir thank hailing frequencies us nothing beam believe sorry ferengi\\
captain picard ships information sensors indicate vessel hailed systems energy\
one would angel meters might elements true enterprise life odin\\
commander time doctor minutes go information outpost ahead riker late\\
years two ago hundred ornara thousand much system seven planets'
#text = str(display_topics1(nmf_model, cv.get_feature_names(), 10))
def generate_wordcloud(text): # optionally add: stopwords=STOPWORDS and change the arg below
#     wordcloud = WordCloud(font_path='/Library/Fonts/Verdana.ttf',
#                           relative_scaling = 0.5,
#                           stopwords = {'to', 'of'} # set or space-separated string
#                           ).generate(text)
#     plt.imshow(wordcloud)
#     plt.axis("off")
#     plt.show()
    wordcloud = WordCloud(background_color="black").generate(text)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    
def generate_wc(ilist):
    for i in range(0, len(ilist)):
        generate_wordcloud(ilist[i])
    
generate_wc(list1)

In [ ]:
#tngspeech['who']=tngspeech['who'].str.strip()

In [ ]:
tngspeech['who'].value_counts()

In [ ]:
#tngspeech.groupby('who').value_counts()

#grouped = tngspeech.groupby('who').value_counts()
#print grouped['Points'].agg(np.mean)
call=tngspeech.groupby(['who'])['post'].count().reset_index().sort_values(by='post',ascending=False).head(20)
call.reset_index(drop=True)

In [ ]:
call.plot.bar(x='who',  y='post', rot=0,fontsize =10,color=(0.2, 0.4, 0.6, 0.6))
plt.xticks(rotation='vertical',fontsize=10)
plt.ylabel('COUNT',fontsize=10)
plt.suptitle('Top 20 characters who speak', fontsize=35)
plt.legend(loc='best', fontsize=10)

# analysis of season 1

In [ ]:
mask=(tngspeech['Season']==1.0)
tngseason1=tngspeech[mask]
tngseason1

In [ ]:
#tngseason1.drop(columns=["component_1","component_2",'component_3','component_4','component_5','component_6','component_7','component_8','component_9','component_10' ], inplace=True)
tngseason1.drop(columns=['0','productionnumber', 'setnames','characters','act','scenenumber',"scenedetails","partnumber",'speechdescription','imdbID' ])

In [ ]:
tngseason1.head()

In [ ]:
#tngseason1=tngspeech[mask]

In [ ]:
tngseason1.drop(columns=['0','productionnumber', 'setnames','characters','act','scenenumber',"scenedetails","partnumber",'speechdescription','imdbRating','imdbID' ], inplace=True)

In [ ]:
tngseason1.head()

In [ ]:
# my_list =['im', 'sorry', 'going', 'sure', 'must', 'first', 'course', tell, ive, whats,thank', 'good', 'still', 'perform', 'lost', 'enterprise', 'ill', 'right','know', 'dont', 'understand', 'think', 'youre', 'even', 'time', 'much', 'cant', 'anything','us', 'would', 'get', 'take', 'well', 'want', 'back', 'need', 'life', 'see','like','yes', 'aye', 'just', 'like', 'way', 'power', 'theyre','may','could', 'something', 'open', 'point', 'show', 'ready']

In [ ]:
stop_english= stopwords.words('english')

In [ ]:
stop_english.extend(my_list)

In [ ]:
cv = CountVectorizer(stop_words=stop_english)

s1=cv.fit_transform(tngseason1['post'])

In [ ]:
import pandas as pd

pd.DataFrame(s1.toarray(), index=tngseason1['post'], columns=cv.get_feature_names()).head(10)

In [ ]:
nmf_model = NMF(10)
doc_topic = nmf_model.fit_transform(s1)

In [ ]:
topic_word1 = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2",'component_3','component_4','component_5','component_6','component_7','component_8','component_9','component_10' ],
             columns = cv.get_feature_names())
topic_word1

In [ ]:
nmf_model.components_

In [ ]:
#5 TOPICS FOR S1
display_topics(nmf_model, cv.get_feature_names(), 10)

# DATA ANALYSIS

In [ ]:
nlp = spacy.load('en', disable=['ner', 'parser'])

In [ ]:
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [ ]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in tngspeech['text'])

In [ ]:
import datetime
import time
import re

t = time.time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time.time() - t) / 60, 2)))

In [ ]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
sent = [row.split() for row in df_clean['clean']]

In [ ]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

In [ ]:
bigram = Phraser(phrases)

In [ ]:
sentences = bigram[sent]

In [ ]:
import collections
word_freq = collections.defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

In [ ]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

In [ ]:
import multiprocessing

from gensim.models import Word2Vec

In [ ]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [ ]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
t = time.time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time.time() - t) / 60, 2)))

In [ ]:
t = time.time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time.time() - t) / 60, 2)))

In [ ]:
w2v_model.init_sims(replace=True)

In [ ]:
w2v_model.wv.most_similar(positive=["picard"])

In [ ]:
w2v_model.wv.most_similar(positive=["jean_luc"])

In [ ]:
w2v_model.wv.most_similar(positive=["riker"])

In [ ]:
w2v_model.wv.most_similar(positive=["data"])

In [ ]:
w2v_model.wv.most_similar(positive=["geordi"])

In [ ]:
w2v_model.wv.most_similar(positive=["crusher"])

In [ ]:
w2v_model.wv.most_similar(positive=["troi"])

In [ ]:
w2v_model.wv.most_similar(positive=["worf"])

In [ ]:
w2v_model.wv.most_similar(positive=["tasha"])

In [ ]:
w2v_model.wv.most_similar(positive=["beverly"])

In [ ]:
w2v_model.wv.most_similar(positive=["wesley"])

In [ ]:
#w2v_model.wv.most_similar(positive=["pulaski"])

In [ ]:
#TSNE Visualizations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=20).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))
    

In [ ]:
[('jean_luc', 0.9754818677902222),
 ('welcome_aboard', 0.942453145980835),
 ('riker', 0.903749406337738),
 ('uss_enterprise', 0.8965575695037842),
 ('william_riker', 0.8842464685440063),
 ('welcome', 0.8726394176483154),
 ('federation_starship', 0.868659496307373),
 ('urgent', 0.867255449295044),
 ('meet', 0.8623436689376831),
 ('admiral', 0.8589075803756714)]

In [ ]:
#Let's compare where the vector representation of Picard, his 10 most similar words from the model, as well as 8 random ones, lies in a 2D graph:

In [ ]:
tsnescatterplot(w2v_model, 'picard', ['data', 'phaser', 'doctor', 'geordi', 'worf', 'enterprise', 'federation', 'troi','beverly','crusher'])

In [ ]:
#This time, let's compare where the vector representation of Picard and his 10 most similar words from the model lies compare to the vector representation of the 10 most dissimilar words to Picard

In [ ]:
tsnescatterplot(w2v_model, 'picard', [i[0] for i in w2v_model.wv.most_similar(negative=["picard"])])

In [ ]:
#Finally, we are going to plot the most similar words to Picard ranked 1st to 10th versus the ones ranked 11th to 20th:

In [ ]:
tsnescatterplot(w2v_model, "picard", [t[0] for t in w2v_model.wv.most_similar(positive=["picard"], topn=20)][10:])

In [ ]:
tngspeech.head()

# vader

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
#brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in tngspeech['text'])
#tngspeech['sentiment'] = np.where(vs['pos']-vs['neg'] > 0, 'positive', 'negative')
analyzer = SentimentIntensityAnalyzer()
vs = analyzer.polarity_scores(tngspeech['post'][0])
# tngspeech['sentiment1']= vs
vs['compound']

In [ ]:
tngspeech['compound'] = tngspeech['text'].map(lambda x: analyzer.polarity_scores(x)['compound'])

In [ ]:
#tngspeech['negative'] = tngspeech['text'].map(lambda x: analyzer.polarity_scores(x)['neg'])

In [ ]:
#tngspeech['positive'] = tngspeech['text'].map(lambda x: analyzer.polarity_scores(x)['pos'])

In [ ]:
#tngspeech['neutral'] = tngspeech['text'].map(lambda x: analyzer.polarity_scores(x)['neu'])

In [ ]:
tngspeech.head(30)

In [ ]:
#tngspeech.drop(columns=['sentiment'],inplace=True)
#tngspeech.drop(columns=['negative','positive' ,'neutral'], inplace=True)

In [ ]:
def sentiment_mapper(x):
    if x >= 0.05:
        result = 'positive'
    elif x <= -0.05:
        result = 'negative'
    else:
        result = 'neutral'
    return result

In [ ]:
tngspeech['compound'].head().map(sentiment_mapper)

In [ ]:
tngspeech.head(30)
tngspeech['sentiment'] = tngspeech['compound'].map(sentiment_mapper)
#positive sentiment: compound score >= 0.05
#neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
#negative sentiment: compound score <= -0.05

In [ ]:
tngspeech.head()

In [ ]:
tngspeech.to_csv('tngdf1.csv')

In [ ]:
filter3=tngspeech.groupby(['Season','sentiment'])['who'].value_counts()
filter3 = filter3.reset_index(name='count')
filter3.sort_values(['Season','sentiment','count'],ascending=False).groupby(['Season','sentiment']).head(20)


In [ ]:
tngspeech.info()

In [ ]:
tngspeech['Released'] =  pd.to_datetime(tngspeech['Released'], format='%Y-%m-%d')

In [ ]:
tngspeech.head()

In [ ]:
filter5=tngspeech.groupby(['Season','episode', 'Released'])['imdbRating'].mean()
filter5 = filter5.reset_index(name='rating')
filter5.sort_values(['Season','Released','episode'],ascending=True).groupby(['Released']).head(20)

In [ ]:
filter5

In [ ]:
filter4=tngspeech.groupby(['who'])['compound'].mean()
filter4 = filter4.reset_index(name='count')
newdf=filter4.sort_values(['who','count'],ascending=False).groupby(['who']).head(20)

In [ ]:
newdf

In [ ]:
mask = newdf['who']=='PULASKI'

In [ ]:
newdf[mask]

In [ ]:
import os
import gensim
model1 = gensim.models.KeyedVectors.load_word2vec_format(tngspeech['post'], binary=True)

In [ ]:
model1.most_similar('picard' ,topn=4)

In [ ]:
import gensim

In [ ]:
tokenized_docs = [gensim.utils.simple_preprocess(d) for d in tngspeech['post']]

In [ ]:
model = gensim.models.Word2Vec(tokenized_docs, size=10, window=2,min_count=1, sg=1)

In [ ]:
model.most_similar('bajoran' ,topn=10)

In [ ]:
model.most_similar(positive=['ferengi', 'klingon'], negative=['man'])

In [ ]:
model.most_similar(positive=['borg', 'vulcan'])

In [ ]:
model.most_similar(positive=['vulcan', 'romulan'])

In [ ]:
model.most_similar(positive=['cardassian', ])

In [ ]:
import scipy.sparse as ss

In [ ]:
# Transform 20 newsgroup data into a sparse matrix
vectorizer = CountVectorizer(stop_words='english', max_features=20000, binary=True)
doc_word = vectorizer.fit_transform(tngspeech['post'])
doc_word = ss.csr_matrix(doc_word)

doc_word.shape # n_docs x m_words

In [ ]:
# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = list(np.asarray(vectorizer.get_feature_names()))

In [ ]:
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words    = [word for ind,word in enumerate(words) if not word.isdigit()]

doc_word.shape # n_docs x m_words

In [ ]:
topic_model = ct.Corex(n_hidden=10, words=words, max_iter=200, verbose=False, seed=1, docs=tngspeech['post'])
topic_model.fit(doc_word, words=words);

In [ ]:
topic_model.get_topics(topic=1, n_words=10)

In [ ]:
# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

In [ ]:
topic_model.get_top_docs(topic=0, n_docs=10)

In [ ]:
tngspeech['post'].iloc[272080]

In [ ]:
model.most_similar(positive=['riker', 'troi'])

In [ ]:
model.most_similar(positive=['ferengi', 'cardassian'])

In [ ]:
model.most_similar(positive=['borg', 'vulcan'])

In [ ]:
model.most_similar(positive=['borg', 'vulcan', 'ferengi','cardassian', 'romulan', 'klingon','bajoran'])